In [27]:
#Taken from CNN-LSTM/Video-Classification-CNN-and-LSTM--master/train_CNN_RNN.py

In [1]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Conv2D,Conv3D, MaxPooling2D, TimeDistributed
from keras import applications
from keras.preprocessing import image 
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications.vgg16 import VGG16
from keras.layers import LSTM
from keras.utils import to_categorical
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import pandas as pd
from glob import glob
from tqdm import tqdm
from scipy.misc import imread, imresize
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms,datasets, models
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
import copy
import time

Using TensorFlow backend.


In [29]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    pinMem = True
else:
    pinMem = False

batch_size = 128

path=r"E:\Documents\Jobs\NP Industry Project - Coddie\Research on Behavioral Teaching\Face Image Datasets\DAiSEE"
traindf=pd.read_csv(path+"/Labels/TrainFrameLabels.csv",dtype=str)
valdf=pd.read_csv(path+"/Labels/ValidationFrameLabels.csv",dtype=str)

In [30]:
# creating an empty list
images = glob(path+"\DataSet\Train\TrainFrames/*.jpg")
trainpart_image = []
train_image = []
trainpart_label = []
train_label = []

# for loop to read and store frames
for i in tqdm(range(traindf.shape[0])): 
    if i % 500 == 0:
        # loading the image and keeping the target size as (224,224,3)
        img = image.load_img(path+"\DataSet\Train\TrainFrames/"+traindf['FrameID'][i], target_size=(224,224,3))
        # converting it to array
        img = image.img_to_array(img)
        # normalizing the pixel value
        img = img/255
        # appending the image to the train_image list
        trainpart_image.append(img)
        trainpart_label.append(float(traindf['Engagement'][i]))
    if i % 62500 == 0:
        train_image.append(trainpart_image)
        train_label.append(trainpart_label)
        trainpart_image=[]
        trainpart_label=[]
        
# converting the list to numpy array
X_train = np.stack(train_image[1:-1])
y_train = np.array(train_label[1:-1])
y_train = to_categorical(y_train)

# shape of the array
print(X_train.shape)
print(y_train.shape)

# creating an empty list
images = glob(path+"\DataSet\Validation\ValidationFrames/*.jpg")
validationpart_image = []
validation_image = []
validationpart_label = []
validation_label = []

# for loop to read and store frames
for i in tqdm(range(valdf.shape[0])):
    if i % 500 == 0:
        # loading the image and keeping the target size as (224,224,3)
        img = image.load_img(path+"\DataSet\Validation\ValidationFrames/"+valdf['FrameID'][i], target_size=(224,224,3))
        # converting it to array
        img = image.img_to_array(img)
        # normalizing the pixel value
        img = img/255
        # appending the image to the train_image list
        validationpart_image.append(img)
        validationpart_label.append(float(valdf['Engagement'][i]))
    if i % 62500 == 0:
        validation_image.append(validationpart_image)
        validation_label.append(validationpart_label)
        validationpart_image=[]
        validationpart_label=[]
        
# converting the list to numpy array
X_val = np.stack(validation_image[1:-1])
y_val = np.array(validation_label[1:-1])
y_val = to_categorical(y_val)

# shape of the array
print(X_val.shape)
print(y_val.shape)

100%|████████████████████████████████████████████████████| 1645782/1645782 [00:14<00:00, 111239.49it/s]


(25, 125, 224, 224, 3)
(25, 125, 4)


100%|███████████████████████████████████████████████████████| 516784/516784 [00:18<00:00, 27809.70it/s]


(7, 125, 224, 224, 3)
(7, 125, 4)


In [33]:
import tensorflow as tf
input_shape = (125, 224, 224, 3)

model = Sequential()
model.add(TimeDistributed(Conv2D(32, (5, 5), padding='same', activation='relu'),input_shape=input_shape))
model.add(TimeDistributed(Conv2D(32, (5, 5), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Dropout(0.5)))
model.add(TimeDistributed(Conv2D(8, (5, 5), padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(8, (5, 5), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Dropout(0.5)))
model.add(TimeDistributed(Flatten()))

model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32, return_sequences=True))
model.add(Dense(4, activation='softmax'))

print(model.summary())

sgd = SGD(lr=0.00005, decay = 1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = [ EarlyStopping(monitor='val_loss', patience=10, verbose=0), ModelCheckpoint('video_1_LSTM_1_1024.h5', monitor='val_loss', save_best_only=True, verbose=0) ]
nb_epoch = 30

model.fit(X_train,y_train,validation_data=(X_val,y_val),batch_size=1, epochs=nb_epoch,callbacks=callbacks,verbose=1)
#model.fit_generator(generator=train_generator(size=size, batch_size=batch_size), steps_per_epoch = 1645782 // 30,epochs = 10, validation_data=validation_generator, validation_steps = 1516784 // 30)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_169 (TimeDi (None, 125, 224, 224, 32) 2432      
_________________________________________________________________
time_distributed_170 (TimeDi (None, 125, 220, 220, 32) 25632     
_________________________________________________________________
time_distributed_171 (TimeDi (None, 125, 110, 110, 32) 0         
_________________________________________________________________
time_distributed_172 (TimeDi (None, 125, 110, 110, 32) 0         
_________________________________________________________________
time_distributed_173 (TimeDi (None, 125, 110, 110, 8)  6408      
_________________________________________________________________
time_distributed_174 (TimeDi (None, 125, 106, 106, 8)  1608      
_________________________________________________________________
time_distributed_175 (TimeDi (None, 125, 53, 53, 8)    0         
__________

In [34]:
model.save("model_1.h5")

In [2]:
model = tf.keras.models.load_model('model_1.h5')

path=r"E:\Documents\Jobs\NP Industry Project - Coddie\Research on Behavioral Teaching\Face Image Datasets\DAiSEE"
testdf=pd.read_csv(path+"/Labels/TestFrameLabels.csv",dtype=str)

# creating an empty list
images = glob(path+"\DataSet\Test\TestFrames/*.jpg")
testpart_image = []
test_image = []
testpart_label = []
test_label = []

# for loop to read and store frames
for i in tqdm(range(testdf.shape[0])):
    if i % 500 == 0:
        # loading the image and keeping the target size as (224,224,3)
        img = image.load_img(path+"\DataSet\Test\TestFrames/"+testdf['FrameID'][i], target_size=(224,224,3))
        # converting it to array
        img = image.img_to_array(img)
        # normalizing the pixel value
        img = img/255
        # appending the image to the train_image list
        testpart_image.append(img)
        testpart_label.append(float(testdf['Engagement'][i]))
    if i % 62500 == 0:
        test_image.append(testpart_image)
        test_label.append(testpart_label)
        testpart_image=[]
        testpart_label=[]
        
# converting the list to numpy array
X_test = np.stack(test_image[1:-1])
y_test = np.array(test_label[1:-1])
y_test = to_categorical(y_test)

# shape of the array
print(X_test.shape)
print(y_test.shape)

100%|███████████████████████████████████████████████████████| 536416/536416 [00:23<00:00, 22607.95it/s]


(7, 125, 224, 224, 3)
(7, 125, 4)


In [3]:
test_loss, test_acc = model.evaluate(X_test,  y_test, batch_size=1, verbose=2)

print("Test Accuracy: " + str(test_acc))
print("Test Loss: " + str(test_loss))

Test Accuracy: 0.5268571376800537
Test Loss: 0.9409289530345372


In [4]:
model.save("model_1test.h5")

'''
def test_on_whole_videos(train_data,train_labels,validation_data,validation_labels):
    parent = os.listdir("/Users/.../video/test")
    #.....................................Testing on whole videos.................................................................
    x = []
    y = []
    count = 0
    output = 0
    count_video = 0
    correct_video = 0
    total_video = 0
    base_model = load_VGG16_model()
    model = train_model(train_data,train_labels,validation_data,validation_labels)
    for video_class in parent[1:]:
        print (video_class)
        child = os.listdir("/Users/.../video/test" + "/" + video_class)
        for class_i in child[1:]:
            sub_child = os.listdir("/Users/.../video/test" + "/" + video_class + "/" + class_i)
            for image_fol in sub_child[1:]:
                if (video_class ==  'class_4' ):
                    if(count%4 == 0):
                        image = imread("/Users/.../video/test" + "/" + video_class + "/" + class_i + "/" + image_fol)
                        image = imresize(image , (224,224))

                        x.append(image)
                        y.append(output)
                        #cv2.imwrite('/Users/.../video/validate/' + video_class + '/' + str(count) + '_' + image_fol,image)
                    count+=1

                else:
                    if(count%4 == 0):
                        image = imread("/Users/.../video/test" + "/" + video_class + "/" + class_i + "/" + image_fol)
                        image = imresize(image , (224,224))
                        x.append(image)
                        y.append(output)
                        #cv2.imwrite('/Users/.../video/validate/' + video_class + '/' + str(count) + '_' + image_fol,image)
                    count+=1
            #correct_video+=1
            x = np.array(x)
            y = np.array(y)
            x_features = base_model.predict(x)
            #np.save(open('feat_' + 'class_' + str(output) + '_' + str(count_video) +'_'  + '.npy','w'),x)

            correct = 0

            answer = model.predict(x_features)
            for i in range(len(answer)):
                if(y[i] == np.argmax(answer[i])):
                    correct+=1
            print (correct,"correct",len(answer))
            total_video+=1
            if(correct>= len(answer)/2):
                correct_video+=1
            x = []
            y = []
            count_video+=1
        output+=1

    print ("correct_video",correct_video,"total_video",total_video)
    print ("The accuracy for video classification of ",total_video, " videos is ", (correct_video/total_video))
    
if __name__ == '__main__':
    train_generator,validation_generator = bring_data_from_directory()
    base_model = load_VGG16_model()
    train_data,train_labels,validation_data,validation_labels = extract_features_and_store(train_generator,validation_generator,base_model)
    train_model(train_data,train_labels,validation_data,validation_labels)
    test_on_whole_videos(train_data,train_labels,validation_data,validation_labels)

'''